In [1]:
import time

def decorated(func):
    cache = {}
    def wrapped(variable, data, plot_type):
        args = str(len(data)) + variable + plot_type
        if args in cache:
            return cache[args]
        else:
            val = func(variable, data, plot_type)
            cache[args] = val
            return val
    return wrapped

@decorated
def some_function(variable, data, plot_type):
    time.sleep(10)
    return variable

In [27]:
from threading import Thread
from pymc3.exceptions import SamplingError
import pandas as pd
import pymc3 as pm

def something():
    d = pd.read_csv("Howell1.csv", sep=";", header=0)
    data = d[d.age >= 18]

    prior_params= {
        'height_mean_mu':178,
        'height_mean_sd':-10,
        'sd_lower':-10,
        'sd_upper':50,
    }
    with pm.Model() as model:
        mu = pm.Normal("mu", mu=prior_params['height_mean_mu'], sd=prior_params['height_mean_sd'])
        sigma = pm.Uniform("sigma", lower=prior_params['sd_lower'], upper=prior_params['sd_upper'])
        height = pm.Normal("height", mu=mu, sd=sigma, observed=data.height)
        pm.sample()
    return model

def a():
    something()

def another():
    try:
        a()
    except Exception as e:
        print('caught')



another()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, mu]


Bad initial energy, check any log probabilities that are inf or -inf, nan or very small:
mu   -inf
caught


In [23]:
kw = {
    'variable':'switchpoint',
    'data':{'mark':10, 'chris':15},
    'plot_type':'same_plot',
}

m = some_function(**kw)
print(m)

switchpoint


In [24]:
str(kw.keys())

"dict_keys(['variable', 'data', 'plot_type'])"

In [ ]:
som

In [35]:
import pymc3 as pm
import theano

# def model_method(data, **prior_params):
#     with pm.Model() as model:
#         mu = pm.Normal("mu", mu=prior_params['height_mean_mu'], sd=prior_params['height_mean_sd'])
#         sigma = pm.Uniform("sigma", lower=prior_params['sd_lower'], upper=prior_params['sd_upper'])
#         height = pm.Normal("height", mu=mu, sd=sigma, observed=data.height)
    # return model

def model_method(data, **prior_params):
    with pm.Model() as model:
        mu = model.Var('mu',prior_params['mu'])
        sigma = pm.Uniform("sigma", lower=prior_params['sd_lower'], upper=prior_params['sd_upper'])
        height = pm.Normal("height", mu=mu, sd=sigma, observed=data.height)
    return model

In [63]:
import panel as pn

# slider for small values less than one. Might me useful if adding in compatimbility for weakly informative priors
s = pn.widgets.FloatSlider(start=0, end=0.1, format=PrintfTickFormatter(format='%.5f'), step = .00001)
pn.serve(s)


Launching server at http://localhost:64514


In [64]:
s.value

0.01644

In [1]:
import pandas as pd

d = pd.read_csv("Howell1.csv", sep=";", header=0)
data = d[d.age >= 18]

# with pm.Model() as normal:
#     mu = pm.Normal("mu", mu=178, sd=prior_params['height_mean_sd'])

# params= {
#     'height_mean_mu':178,
#     'height_mean_sd':20,
#     'sd_lower':0,
#     'sd_upper':50,
# }

params= {
    'mu' : pm.Normal.dist(mu=170, sd=10),
    'sd_lower':0,
    'sd_upper':50,
}

NameError: name 'pm' is not defined

In [37]:
m = model_method(data, **params)

In [38]:
m

In [22]:
with m:
    m.__delattr__('sigma')

In [23]:
m.mu

AttributeError: 'Model' object has no attribute 'mu'

In [24]:
m

In [18]:
m.Var('mu', pm.Normal.dist(mu=170, sd=20))

ValueError: Variable name mu already exists.

In [20]:
m.mu

AttributeError: 'Model' object has no attribute 'mu'

In [67]:
with m:
    mu = pm.Normal('mu', mu=179, sd=20)

ValueError: Variable name mu already exists.

In [61]:
m.mu = pm.Normal.dist(mu=179, sd=20)

In [64]:
m.mu.name = 'mu'

In [65]:
m.mu

In [40]:
m.vars

[mu, sigma_interval__]

In [15]:
mj = m.__latex__()

In [18]:
print("""<script type="math/asciimath"> """ + mj +""""</script>""")

<script type="math/asciimath"> $$
            \begin{array}{rcl}
            \text{mu} &\sim & \text{Normal}(\mathit{mu}=178.0,~\mathit{sigma}=20.0)\\\text{sigma} &\sim & \text{Uniform}(\mathit{lower}=0.0,~\mathit{upper}=50.0)\\\text{height} &\sim & \text{Normal}(\mathit{mu}=\text{mu},~\mathit{sigma}=f(\text{sigma}))
            \end{array}
            $$"</script>


In [6]:
from prior_views.app import app_view

app = app_view(model_method, data, **params)

Launching server at http://localhost:49316


In [ ]:
import arviz as az 

d = az.from_pymc3(
    
)

In [4]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [5]:
J = 8
y = np.array([28,  8, -3,  7, -1,  1, 18, 12])
sigma = np.array([15, 10, 16, 11,  9, 11, 10, 18])

In [8]:
with pm.Model() as hierarchical:

    eta = pm.Normal('eta', 0, 1, shape=J)
    mu = pm.Normal('mu', 0, sd=1e6)
    tau = pm.HalfCauchy('tau', 5)

    theta = pm.Deterministic('theta', mu + tau*eta)

    obs = pm.Normal('obs', theta, sd=sigma, observed=y)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, mu, eta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4 seconds.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.


In [ ]:
posterior = posterior_den

In [ ]:
import arviz as az 

az.plot_density()